# Constant Phase Shift of a Square Wave

A square wave with a fundamental frequency $f_0$
is described within the corresponding period $T_0 = 1 / f_0$ as

\begin{align}
a(t) =
\begin{cases}
1,& 0 < t < \tfrac{T_0}{2}\\
0,& \tfrac{T_0}{2} < t < T_0
\end{cases}
\end{align}

The exponential Fourier series coefficients read

\begin{align}
C_m
&= \frac{1}{T_0}
\int_{0}^{T_0} a(t) \cdot e^{i\frac{2\pi m}{T_0}t} \text{d}t\\
&=
\begin{cases}
-\frac{2i}{m\pi},& \text{$m$ odd}\\
0, & \text{$m$ even}.
\end{cases}
\end{align}

Due to the constant phase of $C_m$,
the Fourier series expansion of the square wave
requires only sine functions,

\begin{align}
a(t) 
= \sum_{m=1, \text{odd}}^{\infty}
\frac{4}{\pi m} \sin\big(\tfrac{2\pi m}{T_0}t\big),
\end{align}

therefore,

\begin{align}
A_{m} =
\begin{cases}
\frac{4}{m \pi}, & \text{$m$ odd}\\
0, & \text{$m$ even}
\end{cases}
\end{align}

If the square wave is filtered with a constant phase shifter,
the individual harmonic components (sine waves)
are shifted by a constant phase angles
irrespective of the order and frequency,

\begin{align}
a_{\varphi}(t)
= \sum_{m=1,\text{odd}}^{\infty}
\frac{4}{\pi m} \sin(\tfrac{2 \pi m }{T_0}t + \varphi),
\end{align}

where $\varphi$ denotes the phase angle of the phase shifter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import util
import importlib
from scipy.signal import freqz, kaiser, fftconvolve as conv
from os.path import join
from matplotlib import cm
%matplotlib inline
importlib.reload(util);

In [ ]:
fs = 44100
f0 = 440
num_periods = 3
duration = num_periods / f0
amplitude = 1
num_partials = 1000
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]
phase_angles = np.linspace(-np.pi, np.pi, num=9, endpoint=True)

fig, ax = plt.subplots(figsize=(8, 8))
voffset_time = 5
for i, phi in enumerate(phase_angles):
    t, s, N = util.square_wave(f0, num_partials, amplitude, duration,
                               fs, phi, modal_window)
    ax.plot(t * 1000, s - i * voffset_time)
    ax.text(num_periods * 1000 / f0, -i * voffset_time,
            r'${:0.0f}^\circ$ ({:0.2f} dB)'
            .format(np.rad2deg(phi),util.db(util.crest_factor(s))), ha='left')
ax.set_xlim(0, num_periods * 1000 / f0)
ax.set_xlabel('$t$ / ms')
ax.grid()
ax.set_title('$f_0 = {}$ Hz, {:0.0f} partials'.format(f0, N));

The phase angles and the crest factor of each signal
are indicated on the right of the curve.

## Make Stimuli

FIR realizations of constant phase shifters exhibit
magnitude roll-offs around $f=0, \tfrac{f_s}{2}$.
The deviation depends on the phase angles $\varphi$.
In practice, the filter order can be increased
so that the spectral distortion is imperceptible.

Windowing (fade-in and fade-out) the time domain signal causes
additional distortion to the spectrum.
For phase shifted square waves, the influence of windowing seems
to vary with the phase angle $\varphi$.

Since the present study is interested in
the audiblity of the constant phase distortion,
the stimuli to be compared should
have no perceivable timbral differences.

In order to cope with this issue,
different signal processing procedures are
attempted in the following.

In [ ]:
out_dir = '../data/stimuli'
suffix = '.wav'

fs = 44100
duration = 0.5
amplitude = 0.25
num_partials = 10
fundamentals = 50, 100, 200
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]
phase_angles = np.linspace(-np.pi, np.pi, num=9, endpoint=True)
fade_periods = 3

fmin, fmax, fnum = 2, 22000, 1000
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum, endpoint=True)

### Method I

1. superimpose phase shifted sine waves
2. apply a window function (fade-in, fade-out)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), ncols=2, nrows=len(fundamentals),
                      sharex='col', sharey='col')
voffset_time = 1
voffset_freq = 20
colors = cm.viridis.colors
viridis = cm.get_cmap('viridis', len(phase_angles)).colors

for i, f0 in enumerate(fundamentals):
    fade_in, fade_out = int(fade_periods * fs / f0), int(fade_periods * fs / f0)
    ax[i, 0].fill_between([0, 1000 * fade_in / fs],
                          [-len(phase_angles) * voffset_time, -len(phase_angles) * voffset_time],
                          [voffset_time, voffset_time], facecolor='gray', alpha=0.25)
    for j, phi in enumerate(phase_angles):
        _, s, _ = util.square_wave(f0, num_partials, amplitude, duration,
                                  fs, phi, modal_window)

        s = util.fade(s, fade_in, fade_out, 'q')
        t = 1000 * np.arange(len(s)) / fs
        _, S = freqz(s, 1, f, fs=fs)
        f_label = 1.1 * f0
        i_label = np.argmin(np.abs(f - f_label))

        ax[i, 0].plot(t, s - j * voffset_time, c=viridis[j])
        ax[i, 1].semilogx(f, util.db(S) - j * voffset_freq, c=viridis[j])
        ax[i, 0].text(0, -j * voffset_time, '{:0.0f}'.format(np.rad2deg(phi)))
        ax[i, 1].text(f_label, util.db(S[i_label]) -j * voffset_freq, '{:0.0f}'.format(np.rad2deg(phi)))
    ax[i, 1].set_xlim(fmin, 11 * f0)
    ax[i, 1].set_ylim(-len(phase_angles) * voffset_freq)
    ax[i, 0].grid()
    ax[i, 1].grid()
ax[i, 0].set_xlim(0, 1000 * (fade_periods + 1) / fundamentals[0])
ax[i, 0].set_xlabel('$t$ / ms')
ax[i, 1].set_xlabel('$f$ / Hz')
ax[i, 1].set_ylabel('Magnitude / dB');

The shaded area in the left column indicates
the length of the fade-in window.

Applying a temporal window (fade-in and fade-out) to phase shifted
square waves results in varying spectral distortions,
which should be avoided in the present study.

In [ ]:
fs = 44100
duration = 0.5
amplitude = 0.2
num_partials = 20
fundamentals = 50 * 2**np.arange(4)
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]
phase_angles = np.linspace(-np.pi, np.pi, num=9, endpoint=True)
fade_periods = 3
num_repetitions = 3

for i, f0 in enumerate(fundamentals):
    fade_in, fade_out = int(fade_periods * fs / f0), int(fade_periods * fs / f0)
    for j, phi in enumerate(phase_angles):
        _, s, _ = util.square_wave(f0, num_partials, amplitude, duration,
                                  fs, phi, modal_window)
        s = util.fade(s, fade_in, fade_out, 'q')

        data_name = 'squarewave_f{:04.1f}_phi{:0.0f}'.format(f0, np.rad2deg(phi))
        if (np.abs(s) > 1).any():
            print('Warning: The amplitude ({:0.2f}) exceeds 1 in {}'
                  .format(np.max(np.abs(s)), data_name))
        sf.write(join(out_dir, data_name + suffix),
                 np.tile(s, (1, num_repetitions))[0], fs, subtype='PCM_24')

### Method II

1. Generate a square wave with no phase shift
2. Apply a window function in the time domain
2. Filter the signal with constant phase shifters

In [ ]:
duration = 0.5
amplitude = 0.2
num_partials = 20
fundamentals = 50 * 2**np.arange(3)
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]
fade_periods = 3
num_repetitions = 3

filter_order = 2**16
half_length = filter_order / 2
beta = 8.6
reference_phase = 0
dphi = np.pi / 4
num_angles = int(2 * np.pi / dphi) + 1
phase_angles = np.linspace(-np.pi, np.pi, num=num_angles, endpoint=True)
phase_angles_deg = np.rad2deg(phase_angles)
phase_shifters = [util.constant_phase_shifter(filter_order, phi, beta=beta)[1]
                  for phi in phase_angles]

fig, ax = plt.subplots(figsize=(15, 15), ncols=2, nrows=len(fundamentals),
                      sharex='col', sharey='col')
voffset_time = 1
voffset_freq = 20
colors = cm.viridis.colors
viridis = cm.get_cmap('viridis', len(phase_angles)).colors
t_label = 1000 * half_length / fs

for i, f0 in enumerate(fundamentals):
    fade_in, fade_out = int(fade_periods * fs / f0), int(fade_periods * fs / f0)
    ax[i, 0].fill_between([t_label, t_label + 1000 * fade_in / fs],
                          [-len(phase_angles) * voffset_time, -len(phase_angles) * voffset_time],
                          [voffset_time, voffset_time], facecolor='gray', alpha=0.25)
    _, s, _ = util.square_wave(f0, num_partials, amplitude, duration,
                                  fs, reference_phase, modal_window)
    s = util.fade(s, fade_in, fade_out, 'q')
    for j, (phi, h) in enumerate(zip(phase_angles, phase_shifters)):
        y = conv(h, s)
        t = 1000 * np.arange(len(y)) / fs
        _, Y = freqz(y, 1, f, fs=fs)
        f_label = 1.1 * f0
        i_label = np.argmin(np.abs(f - f_label))

        ax[i, 0].plot(t, y - j * voffset_time, c=viridis[j])
        ax[i, 1].semilogx(f, util.db(Y) - j * voffset_freq, c=viridis[j])
        ax[i, 0].text(t_label, -j * voffset_time, '{:0.0f}'.format(np.rad2deg(phi)))
        ax[i, 1].text(f_label, util.db(Y[i_label]) -j * voffset_freq, '{:0.0f}'.format(np.rad2deg(phi)))
    ax[i, 1].set_xlim(fmin, 11 * f0)
    ax[i, 1].set_ylim(-len(phase_angles) * voffset_freq)
    ax[i, 0].grid()
    ax[i, 1].grid()
ax[i, 0].set_xlim(t_label, t_label + 1000 * (fade_periods + 1) / fundamentals[0])
ax[i, 0].set_xlabel('$t$ / ms')
ax[i, 1].set_xlabel('$f$ / Hz')
ax[i, 1].set_ylabel('Magnitude / dB');

The spectra of phase shifted square waves are the same.

In [ ]:
for f0 in fundamentals:
    _, s, _ = util.square_wave(f0, num_partials, amplitude, duration, fs,
                                reference_phase, modal_window)
    fade_in, fade_out = int(fade_periods * fs / f0), int(fade_periods * fs / f0)
    s = util.fade(s, fade_in, fade_out, 'q')
    for (phi, h) in zip(phase_angles, phase_shifters):
        y = conv(h, s)
        data_name = 'squarewave_f{:04.1f}_phi{:0.0f}'.format(f0, np.rad2deg(phi))
        if (np.abs(s) > 1).any():
            print('Warning: The amplitude ({:0.2f}) exceeds 1 in {}'
                  .format(np.max(np.abs(s)), data_name))
        sf.write(join(out_dir, data_name + suffix),
                 y, fs, subtype='PCM_24')

### Method III

1. Generate a square wave with no phase shift
2. Apply a window function in the time domain
3. Perform the phase shift in the DFT domain

In [ ]:
duration = 0.5
amplitude = 0.2
num_partials = 100
fundamentals = 50 * 2**np.arange(3)
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]
fade_periods = 3
num_repetitions = 3

reference_phase = 0
dphi = np.pi / 4
num_angles = int(2 * np.pi / dphi) + 1
phase_angles = np.linspace(-np.pi, np.pi, num=num_angles, endpoint=True)
phase_angles_deg = np.rad2deg(phase_angles)

fig, ax = plt.subplots(figsize=(15, 15), ncols=2, nrows=len(fundamentals),
                      sharex='col', sharey='col')
voffset_time = 1
voffset_freq = 20
colors = cm.viridis.colors
viridis = cm.get_cmap('viridis', len(phase_angles)).colors
t_label = 0

for i, f0 in enumerate(fundamentals):
    fade_in, fade_out = int(fade_periods * fs / f0), int(fade_periods * fs / f0)
    _, s, _ = util.square_wave(f0, num_partials, amplitude, duration,
                                  fs, reference_phase, modal_window)
    s = util.fade(s, fade_in, fade_out, 'q')
    S = np.fft.rfft(s)

    ax[i, 0].fill_between([t_label, t_label + 1000 * fade_in / fs],
                          [-len(phase_angles) * voffset_time, -len(phase_angles) * voffset_time],
                          [voffset_time, voffset_time], facecolor='gray', alpha=0.25)
    for j, phi in enumerate(phase_angles):
        y = np.fft.irfft(S * np.exp(1j * phi))
        t = 1000 * np.arange(len(y)) / fs
        _, Y = freqz(y, 1, f, fs=fs)
        f_label = 1.1 * f0
        i_label = np.argmin(np.abs(f - f_label))

        ax[i, 0].plot(t, y - j * voffset_time, c=viridis[j])
        ax[i, 1].semilogx(f, util.db(Y) - j * voffset_freq, c=viridis[j])
        ax[i, 0].text(t_label, -j * voffset_time, '{:0.0f}'.format(np.rad2deg(phi)))
        ax[i, 1].text(f_label, util.db(S[i_label]) -j * voffset_freq, '{:0.0f}'.format(np.rad2deg(phi)))
    ax[i, 1].set_xlim(fmin, 11 * f0)
    ax[i, 1].set_ylim(-len(phase_angles) * voffset_freq)
    ax[i, 0].grid()
    ax[i, 1].grid()
ax[i, 0].set_xlim(t_label, t_label + 1000 * (fade_periods + 1) / fundamentals[0])
ax[i, 0].set_xlabel('$t$ / ms')
ax[i, 1].set_xlabel('$f$ / Hz')
ax[i, 1].set_ylabel('Magnitude / dB');

Even though the DFT length in this example is longer than the FIR phase shifter
in the previous method (method II), the spectral variations are more pronounced.
This is because the ideal phase shift $e^{i \phi}$ is applied to the DFT spectrum.
Recall that an inverse DFT is a periodic signal where the period equals to the DFT length.
Since the time domain response of a phase shift is of infinite length,
its DFT implementation results in a time aliased version of the ideal impulse response.
The influence of the temporal aliasing can be reduced to some extent
by increasing the DFT length, i.e. by zero padding the signal.

A better way to cope with the temporal aliasing is to approximate
the ideal phase shift $e^{i \phi}$ with a truncated Fourier series expansion
and apply it to the DFT spectrum of the input signal.
It can shown that the Fourier coefficient is the reverse
of the FIR coefficients of the impulse response.
This approach is therefore equivalent to the second method.

In [ ]:
duration = 0.5
amplitude = 0.2
num_partials = 100
fundamentals = 50 * 2**np.arange(5)
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]
fade_periods = 3
num_repetitions = 3

reference_phase = 0
dphi = np.pi / 4
num_angles = int(2 * np.pi / dphi) + 1
phase_angles = np.linspace(-np.pi, np.pi, num=num_angles, endpoint=True)
phase_angles_deg = np.rad2deg(phase_angles)

for f0 in fundamentals:
    _, s, _ = util.square_wave(f0, num_partials, amplitude, duration, fs,
                                reference_phase, modal_window)
    fade_in, fade_out = int(fade_periods * fs / f0), int(fade_periods * fs / f0)
    s = util.fade(s, fade_in, fade_out, 'q')
    S = np.fft.rfft(s)
    for (phi, h) in zip(phase_angles, phase_shifters):
        Y = S * np.exp(1j * phi)
        y = np.fft.irfft(Y)
        data_name = 'squarewave_f{:04.1f}_phi{:0.0f}'.format(f0, np.rad2deg(phi))
        if (np.abs(s) > 1).any():
            print('Warning: The amplitude ({:0.2f}) exceeds 1 in {}'
                  .format(np.max(np.abs(s)), data_name))
        sf.write(join(out_dir, data_name + suffix),
                 np.tile(y, (1, num_repetitions))[0], fs, subtype='PCM_24')